<a href="https://colab.research.google.com/github/MEGNEOrnela/NLP_lab1/blob/main/MEGNE_CHOUDJA_Pauline_Ornela_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this second part of the lab, we will implement a language identifier trained on the same data, but using Logistic Regression instead of Naive Bayes.

In [1]:
import io, sys, math
import numpy as np
from collections import defaultdict

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


This function is used to build the dictionary, or vocabulary, which is a mapping from strings (or words) to integers (or indices). This will allow to build vector representations of documents. 

In [2]:
def build_dict(filename, threshold=1):
    fin = io.open(filename, 'r', encoding='utf-8')
    word_dict, label_dict = {}, {}
    counts = defaultdict(lambda: 0)
    for line in fin:
        tokens = line.split()
        label = tokens[0]

        if not label in label_dict:
            label_dict[label] = len(label_dict)

        for w in tokens[1:]:
            counts[w] += 1
            
    for k, v in counts.items():
        if v > threshold:
            word_dict[k] = len(word_dict)
    return word_dict, label_dict

This function is used to load the training dataset, and build vector representations of the training examples. In particular, a document or sentence is represented as a bag of words. Each example correspond to a sparse vector ` x` of dimension `V`, where `V` is the size of the vocabulary. The element `j` of the vector `x` is the number of times the word `j` appears in the document.

In [3]:
def load_data(filename, word_dict, label_dict):
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    dim = len(word_dict)
    for line in fin:
        tokens = line.split()
        label = tokens[0]

        yi = label_dict[label]
        xi = np.zeros(dim)
        for word in tokens[1:]:
            if word in word_dict:
                wid = word_dict[word]
                xi[wid] += 1.0
        data.append((yi, xi))
    return data

First, let's implement the softmax function. Don't forget numerical stability!

In [4]:
def softmax(x):
    ### FILL CODE
    x = x - np.max(x)
    return np.exp(x) / np.sum(np.exp(x), axis=0)

Now, let's implement the main training loop, by using stochastic gradient descent. The function will iterate over the examples of the training set. For each example, we will first compute the loss, before computing the gradient and performing the update.

In [50]:
def sgd(w, data, niter):
    nlabels, dim = w.shape
    for iter in range(niter):
        ### FILL CODE
        for x in data:
          ypred = predict(w,x[1])
          y=x[0]
          #loss
          loss = - np.sum(y * np.log(ypred))
          #gradient
          dw = (1/dim)*np.dot(x[0],(ypred - y))
          w = w - dw
          # print(dw)
    return w

The next function will predict the most probable label corresponding to example `x`, given the trained classifier `w`.

In [54]:
def predict(w, x):
    ## FILL CODE
    p = softmax(np.dot(x,w.T))
    ypred = np.argmax(p)
    return ypred

Finally, this function will compute the accuracy of a trained classifier `w` on a validation set.

In [57]:
def compute_accuracy(w, valid_data):
    ## FILL CODE
    for x in valid_data:
      ypred = predict(w,x[1])
      y = x[0]
      acc = np.sum(y==ypred)/w.shape[1]
      return acc

In [58]:
print("")
print("** Logistic Regression **")
print("")

word_dict, label_dict = build_dict("/content/drive/MyDrive/NLP_Week_1_Labs_2022/session1/train1.txt")
train_data = load_data("/content/drive/MyDrive/NLP_Week_1_Labs_2022/session1/train1.txt", word_dict, label_dict)
valid_data = load_data("/content/drive/MyDrive/NLP_Week_1_Labs_2022/session1/valid1.txt", word_dict, label_dict)

nlabels = len(label_dict)
dim = len(word_dict)
w = np.zeros([nlabels, dim])
w = sgd(w, train_data, 5)
print("")
print("Validation accuracy: %.3f" % compute_accuracy(w, valid_data))
print("")


** Logistic Regression **



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in log
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':



Validation accuracy: 0.000

